In [6]:
import numpy as np                  # Version: 1.24.3
import pandas as pd                 # Version: 1.5.3
import optuna                       # Version: 4.3.0
import optuna.visualization as vis
import torch                        # Version: 2.5.1
import torch.nn as nn
from   torch.utils.data import DataLoader, Dataset
import sklearn                      # Version: 1.3.0
from   sklearn.metrics import accuracy_score, precision_score, recall_score, matthews_corrcoef, roc_auc_score
from   sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt


# Python version: 3.11.4
import os
import random

In [ ]:
notebook_dir = os.getcwd()
parent_dir   = os.path.dirname(notebook_dir)
target_dir   = os.path.join(parent_dir, "Data")

def log_and_print(msg, file_handle):
    print(msg)
    file_handle.write(msg + "\n")

### 1. **HYPERPARAMETER TUNING**

In [ ]:
random.seed(36)
np.random.seed(36)
torch.manual_seed(36)
torch.cuda.manual_seed_all(36)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
g = torch.Generator().manual_seed(36)

s1_train = pd.read_csv(os.path.join(target_dir, "S1_scaled_train.csv"))
s1_val = pd.read_csv(os.path.join(target_dir, "S1_scaled_val.csv"))

log_dir = os.path.join(target_dir, "00_2_validation")

NUMERIC_FEATURES = [
    "NDVI_residual", "NDVI_std", "NDVI_trend", "Solar_de-seasonalized",
    "Temperature_residual", "Humidity_trend", "NDVI_var", "Humidity_residual",
    "NDVI_mean", "Temperature_trend", "Temperature [°C]_max",
    "Solar radiation [Jm2/day]_std", "Relative humidity [%]_mean"
]

CAT_FEATURE = "State"
TARGET_COL  = "fire"

POSSIBLE_LAYER_SIZES = {
    "deep": [256, 128, 64, 32, 16],
    "wide_shallow": [512, 256, 128, 64],
    "small": [128, 64, 32],
    "mid": [256, 128, 64]
}

class WildfireDataset(Dataset):
    def __init__(self, df, sequence_length, only_no_fire=True):
        self.sequence_length = sequence_length
        self.features = NUMERIC_FEATURES
        self.cat_feature = CAT_FEATURE
        grouped = [group for _, group in df.groupby('State', observed=False)]
        self.sequences = []
        self.targets = []
        for g in grouped:
            for i in range(len(g) - sequence_length):
                num_seq = g[self.features].iloc[i:i+sequence_length].values.astype(np.float32)
                cat_seq = g[self.cat_feature].iloc[i:i+sequence_length]
                cat_seq_encoded = cat_seq.astype('category').cat.codes.values.astype(np.float32).reshape(-1,1)
                full_seq = np.hstack([num_seq, cat_seq_encoded])
                label = g[TARGET_COL].iloc[i+sequence_length-1]
                window_labels = g[TARGET_COL].iloc[i:i+sequence_length].values
                if only_no_fire and np.any(window_labels != 0):
                    continue
                self.sequences.append(full_seq)
                self.targets.append(label)
    def __len__(self):
        return len(self.sequences)
    def __getitem__(self, idx):
        return torch.tensor(self.sequences[idx], dtype=torch.float32), torch.tensor(self.targets[idx], dtype=torch.float32)

class LSTMAutoencoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, layer_sizes, num_states, bidirectional=False, dropout=0.3):
        super().__init__()
        self.embedding = nn.Embedding(num_states, embedding_dim)
        self.embedding_dropout = nn.Dropout(dropout)
        enc_input_dim = input_dim + embedding_dim
        self.encoder_layers = nn.ModuleList()
        self.encoder_dropouts = nn.ModuleList()
        for h in layer_sizes:
            self.encoder_layers.append(nn.LSTM(enc_input_dim, h, batch_first=True, bidirectional=bidirectional))
            self.encoder_dropouts.append(nn.Dropout(dropout))
            enc_input_dim = h * (2 if bidirectional else 1)
        dec_input_dim = enc_input_dim
        self.decoder_layers = nn.ModuleList()
        self.decoder_dropouts = nn.ModuleList()
        for h in reversed(layer_sizes[:-1]):
            self.decoder_layers.append(nn.LSTM(dec_input_dim, h, batch_first=True))
            self.decoder_dropouts.append(nn.Dropout(dropout))
            dec_input_dim = h
        self.output_layer = nn.Linear(dec_input_dim, input_dim + embedding_dim)
        self.tanh = nn.Tanh()
    def forward(self, x):
        state_idx = x[..., -1].long()
        emb = self.embedding(state_idx)
        emb = self.embedding_dropout(emb)
        seq = torch.cat([x[..., :-1], emb], dim=-1)
        out = seq
        for lstm, dropout in zip(self.encoder_layers, self.encoder_dropouts):
            out, _ = lstm(out)
            out = dropout(self.tanh(out))
        for lstm, dropout in zip(self.decoder_layers, self.decoder_dropouts):
            out, _ = lstm(out)
            out = dropout(self.tanh(out))
        decoded = self.output_layer(out)
        decoded = self.tanh(decoded)
        return decoded

def train_one_epoch(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0
    embed_dim = model.embedding.embedding_dim
    for x, _ in dataloader:
        optimizer.zero_grad()
        x_hat = model(x)
        x_hat_trimmed = x_hat[:, :, :-embed_dim]
        x_trimmed = x[:, :, :-1]
        loss = criterion(x_hat_trimmed, x_trimmed)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

def evaluate_model(model, dataloader):
    model.eval()
    all_labels, all_scores = [], []
    embed_dim = model.embedding.embedding_dim
    with torch.no_grad():
        for x, y in dataloader:
            x_hat = model(x)
            x_hat_trimmed = x_hat[:, :, :-embed_dim]
            x_trimmed = x[:, :, :-1]
            re = ((x_hat_trimmed - x_trimmed) ** 2).mean(dim=(1,2))
            all_scores.extend(re.numpy())
            all_labels.extend(y.numpy())
    return np.array(all_scores), np.array(all_labels)

def compute_metrics(y_true, y_scores, threshold=None):
    if threshold is None:
        threshold = np.percentile(y_scores[y_true == 0], 95)
    y_pred = (y_scores > threshold).astype(int)
    metrics = {
        "ACC": accuracy_score(y_true, y_pred),
        "PRE": precision_score(y_true, y_pred, zero_division=0),
        "REC": recall_score(y_true, y_pred, zero_division=0),
        "MCC": matthews_corrcoef(y_true, y_pred),
        "AUC": roc_auc_score(y_true, y_scores),
    }
    return metrics, threshold

def find_best_f1_threshold(y_true, y_scores):
    precision, recall, thresholds = precision_recall_curve(y_true, y_scores)
    f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
    best_idx = np.argmax(f1_scores)
    return thresholds[best_idx], f1_scores[best_idx]

#### **Model A 30-Day LSTM-AE**

In [ ]:
# Optuna objective function
def objective(trial):
    is_bilstm = False
    sequence_length = 30

    architecture_choice = trial.suggest_categorical("architecture", list(POSSIBLE_LAYER_SIZES.keys()))
    layer_sizes = POSSIBLE_LAYER_SIZES[architecture_choice]
    
    dropout = trial.suggest_float("dropout", 0.1, 0.5, step=0.05)
    embedding_dim = trial.suggest_int("embedding_dim", 2, 8)
    learning_rate = trial.suggest_float("lr", 1e-4, 1e-2, log=True)

    train_ds = WildfireDataset(s1_train, sequence_length, only_no_fire=True)
    val_ds = WildfireDataset(s1_val, sequence_length, only_no_fire=False)
    train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, generator=g)
    val_loader = DataLoader(val_ds, batch_size=64, shuffle=False)

    model = LSTMAutoencoder(input_dim=len(NUMERIC_FEATURES),
                             embedding_dim=embedding_dim,
                             layer_sizes=layer_sizes,
                             num_states=s1_train[CAT_FEATURE].nunique(),
                             bidirectional=is_bilstm,
                             dropout=dropout)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    for epoch in range(1, 21):
        loss = train_one_epoch(model, train_loader, optimizer, criterion)

    val_scores, val_labels = evaluate_model(model, val_loader)
    threshold, _ = find_best_f1_threshold(val_labels, val_scores)
    metrics, _ = compute_metrics(val_labels, val_scores, threshold)

    trial_msg = (f"Trial {trial.number}: "
                 f"MCC={metrics['MCC']:.4f}, REC={metrics['REC']:.4f}, AUC={metrics['AUC']:.4f}, "
                 f"Params={trial.params}")
    
    log_path = os.path.join(log_dir, "LSTM-AE_optuna_search_log.txt")
    with open(log_path, "a") as f:
        log_and_print(trial_msg, f)
    
    return metrics["MCC"]

# Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Save best trial
best_params_path = os.path.join(log_dir, "LSTM-AE_best_trial_params.txt")
with open(best_params_path, "w") as f:
    f.write(str(study.best_trial.params))

print("Best trial parameters saved to:", best_params_path)

fig1 = vis.plot_optimization_history(study)
fig1.show()
fig2 = vis.plot_param_importances(study)
fig2.show()
fig3 = vis.plot_parallel_coordinate(study)
fig3.show()
fig4 = vis.plot_contour(study)
fig4.show()

fig1.write_html(os.path.join(log_dir, "LSTM-AE_optuna_optimization_history.html"))
fig2.write_html(os.path.join(log_dir, "LSTM-AE_optuna_param_importances.html"))
fig3.write_html(os.path.join(log_dir, "LSTM-AE_optuna_parallel_coordinate.html"))
fig4.write_html(os.path.join(log_dir, "LSTM-AE_optuna_contour.html"))

[I 2025-04-26 16:19:49,185] A new study created in memory with name: no-name-9d0fff45-e779-4a9a-af1f-25c658762a88
[I 2025-04-26 16:24:34,259] Trial 0 finished with value: 0.16886016667262346 and parameters: {'architecture': 'mid', 'dropout': 0.45000000000000007, 'embedding_dim': 2, 'lr': 0.0038685195468653334}. Best is trial 0 with value: 0.16886016667262346.


Trial 0: MCC=0.1689, REC=0.6941, AUC=0.6492, Params={'architecture': 'mid', 'dropout': 0.45000000000000007, 'embedding_dim': 2, 'lr': 0.0038685195468653334}


[I 2025-04-26 16:29:18,438] Trial 1 finished with value: 0.2836189600280694 and parameters: {'architecture': 'mid', 'dropout': 0.30000000000000004, 'embedding_dim': 3, 'lr': 0.0005737249773857136}. Best is trial 1 with value: 0.2836189600280694.


Trial 1: MCC=0.2836, REC=0.3257, AUC=0.6944, Params={'architecture': 'mid', 'dropout': 0.30000000000000004, 'embedding_dim': 3, 'lr': 0.0005737249773857136}


[I 2025-04-26 16:36:58,259] Trial 2 finished with value: 0.1653700122405941 and parameters: {'architecture': 'deep', 'dropout': 0.5, 'embedding_dim': 2, 'lr': 0.001947152099641176}. Best is trial 1 with value: 0.2836189600280694.


...

Trial 47: MCC=0.3729, REC=0.5658, AUC=0.8102, Params={'architecture': 'deep', 'dropout': 0.5, 'embedding_dim': 3, 'lr': 0.0003405365203550537}


[I 2025-04-26 21:35:26,429] Trial 48 finished with value: 0.19976144010814986 and parameters: {'architecture': 'small', 'dropout': 0.4, 'embedding_dim': 4, 'lr': 0.0004749243083503763}. Best is trial 19 with value: 0.4461454940104176.


Trial 48: MCC=0.1998, REC=0.4704, AUC=0.6996, Params={'architecture': 'small', 'dropout': 0.4, 'embedding_dim': 4, 'lr': 0.0004749243083503763}


[I 2025-04-26 21:41:22,719] Trial 49 finished with value: 0.33602106545699706 and parameters: {'architecture': 'deep', 'dropout': 0.15000000000000002, 'embedding_dim': 6, 'lr': 0.0002622573690009186}. Best is trial 19 with value: 0.4461454940104176.


Trial 49: MCC=0.3360, REC=0.5888, AUC=0.8121, Params={'architecture': 'deep', 'dropout': 0.15000000000000002, 'embedding_dim': 6, 'lr': 0.0002622573690009186}
Best trial parameters saved to: c:\Users\tamar\Documents\Thesis 2025\Data\00_2_validation\LSTM-AE_best_trial_params.txt


#### **Model A 14-Day Bi-LSTM-AE**

In [ ]:
# Optuna objective function
def objective(trial):
    is_bilstm = True
    sequence_length = 14

    architecture_choice = trial.suggest_categorical("architecture", list(POSSIBLE_LAYER_SIZES.keys()))
    layer_sizes = POSSIBLE_LAYER_SIZES[architecture_choice]
    
    dropout = trial.suggest_float("dropout", 0.1, 0.5, step=0.05)
    embedding_dim = trial.suggest_int("embedding_dim", 2, 8)
    learning_rate = trial.suggest_float("lr", 1e-4, 1e-2, log=True)

    train_ds = WildfireDataset(s1_train, sequence_length, only_no_fire=True)
    val_ds = WildfireDataset(s1_val, sequence_length, only_no_fire=False)
    train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, generator=g)
    val_loader = DataLoader(val_ds, batch_size=64, shuffle=False)

    model = LSTMAutoencoder(input_dim=len(NUMERIC_FEATURES),
                             embedding_dim=embedding_dim,
                             layer_sizes=layer_sizes,
                             num_states=s1_train[CAT_FEATURE].nunique(),
                             bidirectional=is_bilstm,
                             dropout=dropout)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    for epoch in range(1, 21):
        loss = train_one_epoch(model, train_loader, optimizer, criterion)

    val_scores, val_labels = evaluate_model(model, val_loader)
    threshold, _ = find_best_f1_threshold(val_labels, val_scores)
    metrics, _ = compute_metrics(val_labels, val_scores, threshold)

    trial_msg = (f"Trial {trial.number}: "
                 f"MCC={metrics['MCC']:.4f}, REC={metrics['REC']:.4f}, AUC={metrics['AUC']:.4f}, "
                 f"Params={trial.params}")
    
    log_path = os.path.join(log_dir, "Bi-LSTM-AE_optuna_search_log.txt")
    with open(log_path, "a") as f:
        log_and_print(trial_msg, f)
    
    return metrics["MCC"]

# Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Save best trial
best_params_path = os.path.join(log_dir, "Bi-LSTM-AE_best_trial_params.txt")
with open(best_params_path, "w") as f:
    f.write(str(study.best_trial.params))

print("Best trial parameters saved to:", best_params_path)

fig1 = vis.plot_optimization_history(study)
fig1.show()
fig2 = vis.plot_param_importances(study)
fig2.show()
fig3 = vis.plot_parallel_coordinate(study)
fig3.show()
fig4 = vis.plot_contour(study)
fig4.show()

fig1.write_html(os.path.join(log_dir, "Bi-LSTM-AE_optuna_optimization_history.html"))
fig2.write_html(os.path.join(log_dir, "Bi-LSTM-AE_optuna_param_importances.html"))
fig3.write_html(os.path.join(log_dir, "Bi-LSTM-AE_optuna_parallel_coordinate.html"))
fig4.write_html(os.path.join(log_dir, "Bi-LSTM-AE_optuna_contour.html"))


[I 2025-04-26 22:07:06,501] A new study created in memory with name: no-name-5f388879-2149-4cb6-a44b-9a1d0811018d
[I 2025-04-26 22:22:12,634] Trial 0 finished with value: 0.10103309665326823 and parameters: {'architecture': 'wide_shallow', 'dropout': 0.25, 'embedding_dim': 4, 'lr': 0.000303702813697569}. Best is trial 0 with value: 0.10103309665326823.


Trial 0: MCC=0.1010, REC=0.6019, AUC=0.5750, Params={'architecture': 'wide_shallow', 'dropout': 0.25, 'embedding_dim': 4, 'lr': 0.000303702813697569}


[I 2025-04-26 22:27:29,162] Trial 1 finished with value: 0.15989988976305156 and parameters: {'architecture': 'mid', 'dropout': 0.15000000000000002, 'embedding_dim': 6, 'lr': 0.0002938689826497611}. Best is trial 1 with value: 0.15989988976305156.


Trial 1: MCC=0.1599, REC=0.5287, AUC=0.5924, Params={'architecture': 'mid', 'dropout': 0.15000000000000002, 'embedding_dim': 6, 'lr': 0.0002938689826497611}


[I 2025-04-26 22:30:23,762] Trial 2 finished with value: 0.1762982557456941 and parameters: {'architecture': 'small', 'dropout': 0.25, 'embedding_dim': 6, 'lr': 0.00765161529441038}. Best is trial 2 with value: 0.1762982557456941.


...

Trial 47: MCC=0.1490, REC=0.5223, AUC=0.5748, Params={'architecture': 'small', 'dropout': 0.1, 'embedding_dim': 6, 'lr': 0.0022303307025457455}


[I 2025-04-27 04:32:04,137] Trial 48 finished with value: 0.33929308935155034 and parameters: {'architecture': 'wide_shallow', 'dropout': 0.4, 'embedding_dim': 8, 'lr': 0.0084202194089088}. Best is trial 11 with value: 0.3736816022455939.


Trial 48: MCC=0.3393, REC=0.3439, AUC=0.7951, Params={'architecture': 'wide_shallow', 'dropout': 0.4, 'embedding_dim': 8, 'lr': 0.0084202194089088}


[I 2025-04-27 04:39:42,987] Trial 49 finished with value: 0.2792549350737711 and parameters: {'architecture': 'deep', 'dropout': 0.45000000000000007, 'embedding_dim': 7, 'lr': 0.0007642847052328851}. Best is trial 11 with value: 0.3736816022455939.


Trial 49: MCC=0.2793, REC=0.7548, AUC=0.7523, Params={'architecture': 'deep', 'dropout': 0.45000000000000007, 'embedding_dim': 7, 'lr': 0.0007642847052328851}
Best trial parameters saved to: c:\Users\tamar\Documents\Thesis 2025\Data\00_2_validation\Bi-LSTM-AE_best_trial_params.txt


### **2. VALIDATION AND TESTING**

#### **Models A: Validate and Test**

In [ ]:
s1_test = pd.read_csv(os.path.join(target_dir, "S1_scaled_test.csv"))
s2_test = pd.read_csv(os.path.join(target_dir, "S2_scaled_test.csv"))
test_log_dir = os.path.join(target_dir, "00_3_testing")
val_log_dir  = os.path.join(target_dir, "00_2_validation", "Validation_results")

NUM_RUNS = 6 # number of repetitions to create a distribution
def set_random_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    g = torch.Generator().manual_seed(seed)
    return g

smoothing_alpha = 0.1
patience, min_epoch = 10, 15

modelAB         = "A"
# Reload best parameters
is_bilstm       = False
sequence_length = 30
dropout         = 0.1
embedding_dim   = 8
layer_sizes     = [256, 128, 64, 32, 16]
learning_rate   = 0.0008188148579607131

# Perform multiple training + evaluation runs
all_val_metrics   = []
all_test_metrics  = []
all_test2_metrics = []

for run_idx in range(NUM_RUNS):
    print(f"\n========== Run {run_idx + 1} / {NUM_RUNS} ==========")
    seed = 36 + (run_idx * 10)
    g = set_random_seeds(seed)

    train_ds = WildfireDataset(s1_train, sequence_length, only_no_fire=True)
    val_ds = WildfireDataset(s1_val, sequence_length, only_no_fire=False)
    train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, generator=g)
    val_loader = DataLoader(val_ds, batch_size=64, shuffle=False)
    test_ds = WildfireDataset(s1_test, sequence_length, only_no_fire=False)
    test_loader = DataLoader(test_ds, batch_size=64, shuffle=False)
    test2_ds = WildfireDataset(s2_test, sequence_length, only_no_fire=False)
    test2_loader = DataLoader(test2_ds, batch_size=64, shuffle=False)

    # Create a fresh model for each run
    model = LSTMAutoencoder(
        input_dim=len(NUMERIC_FEATURES),
        embedding_dim=embedding_dim,
        layer_sizes=layer_sizes,
        num_states=s1_train[CAT_FEATURE].nunique(),
        bidirectional=is_bilstm,
        dropout=dropout
    )
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    best_score, smoothed_score, smoothed_best_score = -float('inf'), None, -float('inf')
    counter = 0

    # TRAINING AND VALIDATION
    for epoch in range(1, 201):
        loss = train_one_epoch(model, train_loader, optimizer, criterion)
        val_scores, val_labels = evaluate_model(model, val_loader)
        threshold, _ = find_best_f1_threshold(val_labels, val_scores)
        val_metrics, _ = compute_metrics(val_labels, val_scores, threshold)

        val_loss = np.mean(val_scores)
        mcc = val_metrics["MCC"]
        rec = val_metrics["REC"]
        pre = val_metrics["PRE"]
        auc_score = val_metrics["AUC"]

        scaled_mcc = (mcc + 1) / 2
        scaled_auc = (auc_score - 0.5) * 2
        combo_score = 0.6 * scaled_mcc + 0.05 * rec + 0.05 * pre + 0.05 * scaled_auc

        if smoothed_score is None:
            smoothed_score = combo_score
        else:
            smoothed_score = (smoothing_alpha * combo_score + (1 - smoothing_alpha) * smoothed_score)

        if combo_score > best_score or smoothed_score > smoothed_best_score:
            smoothed_best_score = smoothed_score
            counter = 0
            if combo_score > best_score:
                best_score = combo_score
                best_val_scores = val_scores
                best_val_labels = val_labels
                best_threshold = threshold
                best_val_metrics = val_metrics
                model_path = os.path.join(val_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_model.pth")
                torch.save(model.state_dict(), model_path)
                np.save(os.path.join(val_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_val_labels.npy"), best_val_labels)
                np.save(os.path.join(val_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_val_scores.npy"), best_val_scores)
                np.save(os.path.join(val_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_val_preds.npy"), (np.array(best_val_scores) > best_threshold).astype(int))
        elif epoch >= min_epoch:
            counter += 1
            if counter >= patience:
                print(f"Early stopping at epoch {epoch} (no improvement after {patience} epochs)")
                break

    # Save metrics for this run
    all_val_metrics.append(best_val_metrics)

    run_metrics_path = os.path.join(val_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_metrics.txt")
    with open(run_metrics_path, 'w') as f:
        for k, v in best_val_metrics.items():
            if k in ["ACC", "PRE", "REC"]:
                f.write(f"{k}: {v * 100:.2f}%\n")
            else:
                f.write(f"{k}: {v:.3f}\n")
        f.write(f"Best Threshold: {best_threshold:.5f}\n")

    # Reload the best saved model before testing
    model.load_state_dict(torch.load(model_path, weights_only=True))

    # TESTING
    model.eval()
    test_scores, test_labels = evaluate_model(model, test_loader)
    test_threshold = best_threshold # reuse the one selected on validation
    test_metrics, _ = compute_metrics(test_labels, test_scores, test_threshold)

    all_test_metrics.append(test_metrics)

    test_metrics_path = os.path.join(test_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_test_metrics.txt")
    with open(test_metrics_path, 'w') as f:
        for k, v in test_metrics.items():
            if k in ["ACC", "PRE", "REC"]:
                f.write(f"{k}: {v * 100:.2f}%\n")
            else:
                f.write(f"{k}: {v:.3f}\n")

    test_preds = (np.array(test_scores) > test_threshold).astype(int)
    np.save(os.path.join(test_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_test_labels.npy"), test_labels)
    np.save(os.path.join(test_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_test_scores.npy"), test_scores)
    np.save(os.path.join(test_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_test_preds.npy"), test_preds)

    # TESTING MODEL A ON SET 2
    if modelAB == "A":
        model.eval()
        test_scores, test_labels = evaluate_model(model, test2_loader)
        test_threshold = best_threshold  # reuse the one selected on validation
        test_metrics, _ = compute_metrics(test_labels, test_scores, test_threshold)

        all_test2_metrics.append(test_metrics)

        test_metrics_path = os.path.join(test_log_dir, f"{modelAB}_on_S2_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_test_metrics.txt")
        with open(test_metrics_path, 'w') as f:
            for k, v in test_metrics.items():
                if k in ["ACC", "PRE", "REC"]:
                    f.write(f"{k}: {v * 100:.2f}%\n")
                else:
                    f.write(f"{k}: {v:.3f}\n")

        test_preds = (np.array(test_scores) > test_threshold).astype(int)
        np.save(os.path.join(test_log_dir, f"{modelAB}_on_S2_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_test_labels.npy"), test_labels)
        np.save(os.path.join(test_log_dir, f"{modelAB}_on_S2_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_test_scores.npy"), test_scores)
        np.save(os.path.join(test_log_dir, f"{modelAB}_on_S2_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_test_preds.npy"), test_preds)

# AFTER ALL RUNS
# Compute mean and std for each metric
metrics_summary = {}
for key in all_val_metrics[0].keys():
    values = [metrics[key] for metrics in all_val_metrics]
    metrics_summary[key + '_mean'] = np.mean(values)
    metrics_summary[key + '_std'] = np.std(values)

metrics_summary_path = os.path.join(val_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_final_validation_summary.txt")
with open(metrics_summary_path, 'w') as f:
    for k, v in metrics_summary.items():
        f.write(f"{k}: {v:.3f}\n")

print("\n========== Final Averaged Validation Metrics ==========")
for k, v in metrics_summary.items():
    print(f"{k}: {v:.3f}")

metrics_summary = {}
for key in all_test_metrics[0].keys():
    values = [metrics[key] for metrics in all_test_metrics]
    metrics_summary[key + '_mean'] = np.mean(values)
    metrics_summary[key + '_std'] = np.std(values)

metrics_summary_path = os.path.join(test_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_final_test_summary.txt")
with open(metrics_summary_path, 'w') as f:
    for k, v in metrics_summary.items():
        f.write(f"{k}: {v:.3f}\n")

print("\n========== Final Averaged test Metrics ==========")
for k, v in metrics_summary.items():
    print(f"{k}: {v:.3f}")

if modelAB == "A":
    metrics_summary = {}
    for key in all_test2_metrics[0].keys():
        values = [metrics[key] for metrics in all_test2_metrics]
        metrics_summary[key + '_mean'] = np.mean(values)
        metrics_summary[key + '_std'] = np.std(values)

    metrics_summary_path = os.path.join(test_log_dir, f"{modelAB}_on_S2_{'Bi-' if is_bilstm else ''}LSTM-AE_final_test_summary.txt")
    with open(metrics_summary_path, 'w') as f:
        for k, v in metrics_summary.items():
            f.write(f"{k}: {v:.3f}\n")

    print("\n========== Final Averaged test Metrics ==========")
    for k, v in metrics_summary.items():
        print(f"{k}: {v:.3f}")


========== Run 1 / 6 ==========
Early stopping at epoch 24 (no improvement after 10 epochs)

========== Run 2 / 6 ==========
Early stopping at epoch 34 (no improvement after 10 epochs)

========== Run 3 / 6 ==========
Early stopping at epoch 61 (no improvement after 10 epochs)

========== Run 4 / 6 ==========
Early stopping at epoch 55 (no improvement after 10 epochs)

========== Run 5 / 6 ==========
Early stopping at epoch 24 (no improvement after 10 epochs)

========== Run 6 / 6 ==========
Early stopping at epoch 48 (no improvement after 10 epochs)

========== Final Averaged Validation Metrics ==========
ACC_mean: 0.860
ACC_std: 0.021
PRE_mean: 0.416
PRE_std: 0.061
REC_mean: 0.614
REC_std: 0.062
MCC_mean: 0.428
MCC_std: 0.036
AUC_mean: 0.844
AUC_std: 0.017

========== Final Averaged test Metrics ==========
ACC_mean: 0.792
ACC_std: 0.034
PRE_mean: 0.326
PRE_std: 0.045
REC_mean: 0.831
REC_std: 0.035
MCC_mean: 0.431
MCC_std: 0.034
AUC_mean: 0.878
AUC_std: 0.011

========== Final Averag

In [ ]:
# VALIDATING MODEL A ON SET 2

s2_val = pd.read_csv(os.path.join(target_dir, "S2_scaled_val.csv"))

all_val2_metrics = []

modelAB         = "A"
num_states      = 4 # as used during training
# Reload best parameters
is_bilstm       = False
sequence_length = 30
dropout         = 0.1
embedding_dim   = 8
layer_sizes     = [256, 128, 64, 32, 16]
learning_rate   = 0.0008188148579607131

for run_idx in range(NUM_RUNS):
    seed = 36 + (run_idx * 10)
    g = set_random_seeds(seed)

    val2_ds = WildfireDataset(s2_val, sequence_length, only_no_fire=False)
    val2_loader = DataLoader(val2_ds, batch_size=64, shuffle=False)

    # Reload the best saved model before validation
    model_path = os.path.join(val_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_model.pth")
    
    model = LSTMAutoencoder(
        input_dim=len(NUMERIC_FEATURES), 
        embedding_dim=embedding_dim,
        layer_sizes=layer_sizes,
        num_states=num_states,
        bidirectional=is_bilstm,
        dropout=dropout
    )

    model.load_state_dict(torch.load(model_path, weights_only=True))
    model.eval()

    # Load best threshold from metrics file
    threshold_path = os.path.join(val_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_metrics.txt")
    with open(threshold_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            if "Best Threshold" in line:
                val_threshold = float(line.strip().split(":")[1])
                break

    val_scores, val_labels = evaluate_model(model, val2_loader)
    val_metrics, _ = compute_metrics(val_labels, val_scores, val_threshold)

    all_val2_metrics.append(val_metrics)

    val_metrics_path = os.path.join(val_log_dir, f"{modelAB}_on_S2_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_val_metrics.txt")
    with open(val_metrics_path, 'w') as f:
        for k, v in val_metrics.items():
            if k in ["ACC", "PRE", "REC"]:
                f.write(f"{k}: {v * 100:.2f}%\n")
            else:
                f.write(f"{k}: {v:.3f}\n")

    val_preds = (np.array(val_scores) > val_threshold).astype(int)
    np.save(os.path.join(val_log_dir, f"{modelAB}_on_S2_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_val_labels.npy"), val_labels)
    np.save(os.path.join(val_log_dir, f"{modelAB}_on_S2_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_val_scores.npy"), val_scores)
    np.save(os.path.join(val_log_dir, f"{modelAB}_on_S2_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_val_preds.npy"), val_preds)

# AFTER ALL RUNS
# Compute mean and std for each metric
metrics_summary = {}
for key in all_val2_metrics[0].keys():
    values = [metrics[key] for metrics in all_val2_metrics]
    metrics_summary[key + '_mean'] = np.mean(values)
    metrics_summary[key + '_std'] = np.std(values)

metrics_summary_path = os.path.join(val_log_dir, f"{modelAB}_on_S2_{'Bi-' if is_bilstm else ''}LSTM-AE_final_validation_summary.txt")
with open(metrics_summary_path, 'w') as f:
    for k, v in metrics_summary.items():
        f.write(f"{k}: {v:.3f}\n")

print("\n========== Final Averaged Validation Metrics ==========")
for k, v in metrics_summary.items():
    print(f"{k}: {v:.3f}")


========== Final Averaged Validation Metrics ==========
ACC_mean: 0.660
ACC_std: 0.038
PRE_mean: 0.289
PRE_std: 0.014
REC_mean: 0.689
REC_std: 0.140
MCC_mean: 0.265
MCC_std: 0.051
AUC_mean: 0.723
AUC_std: 0.027


In [ ]:
s1_test = pd.read_csv(os.path.join(target_dir, "S1_scaled_test.csv"))
s2_test = pd.read_csv(os.path.join(target_dir, "S2_scaled_test.csv"))
test_log_dir = os.path.join(target_dir, "00_3_testing")
val_log_dir = os.path.join(target_dir, "00_2_validation", "Validation_results")

NUM_RUNS = 6 # number of repetitions to create a distribution
def set_random_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    g = torch.Generator().manual_seed(seed)
    return g

smoothing_alpha = 0.1
patience, min_epoch = 10, 15

modelAB         = "A"
# 1. Reload best parameters
is_bilstm       = True
sequence_length = 14
dropout         = 0.4
embedding_dim   = 2
layer_sizes     = [256, 128, 64, 32, 16]
learning_rate   = 0.006161330774897911

# Perform multiple training + evaluation runs
all_val_metrics   = []
all_test_metrics  = []
all_test2_metrics = []

for run_idx in range(NUM_RUNS):
    print(f"\n========== Run {run_idx + 1} / {NUM_RUNS} ==========")
    seed = 36 + (run_idx * 10)
    g = set_random_seeds(seed)

    # Prepare datasets
    train_ds = WildfireDataset(s1_train, sequence_length, only_no_fire=True)
    val_ds = WildfireDataset(s1_val, sequence_length, only_no_fire=False)
    train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, generator=g)
    val_loader = DataLoader(val_ds, batch_size=64, shuffle=False)
    test_ds = WildfireDataset(s1_test, sequence_length, only_no_fire=False)
    test_loader = DataLoader(test_ds, batch_size=64, shuffle=False)
    test2_ds = WildfireDataset(s2_test, sequence_length, only_no_fire=False)
    test2_loader = DataLoader(test2_ds, batch_size=64, shuffle=False)

    # Create a fresh model for each run
    model = LSTMAutoencoder(
        input_dim=len(NUMERIC_FEATURES),
        embedding_dim=embedding_dim,
        layer_sizes=layer_sizes,
        num_states=s1_train[CAT_FEATURE].nunique(),
        bidirectional=is_bilstm,
        dropout=dropout
    )
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    best_score, smoothed_score, smoothed_best_score = -float('inf'), None, -float('inf')
    counter = 0

    # TRAINING AND VALIDATION
    for epoch in range(1, 201):
        loss = train_one_epoch(model, train_loader, optimizer, criterion)
        val_scores, val_labels = evaluate_model(model, val_loader)
        threshold, _ = find_best_f1_threshold(val_labels, val_scores)
        val_metrics, _ = compute_metrics(val_labels, val_scores, threshold)

        val_loss = np.mean(val_scores)
        mcc = val_metrics["MCC"]
        rec = val_metrics["REC"]
        pre = val_metrics["PRE"]
        auc_score = val_metrics["AUC"]

        scaled_mcc = (mcc + 1) / 2
        scaled_auc = (auc_score - 0.5) * 2
        combo_score = 0.6 * scaled_mcc + 0.05 * rec + 0.05 * pre + 0.05 * scaled_auc

        if smoothed_score is None:
            smoothed_score = combo_score
        else:
            smoothed_score = (smoothing_alpha * combo_score + (1 - smoothing_alpha) * smoothed_score)

        if combo_score > best_score or smoothed_score > smoothed_best_score:
            smoothed_best_score = smoothed_score
            counter = 0
            if combo_score > best_score:
                best_score = combo_score
                best_val_scores = val_scores
                best_val_labels = val_labels
                best_threshold = threshold
                best_val_metrics = val_metrics
                model_path = os.path.join(val_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_model.pth")
                torch.save(model.state_dict(), model_path)
                np.save(os.path.join(val_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_val_labels.npy"), best_val_labels)
                np.save(os.path.join(val_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_val_scores.npy"), best_val_scores)
                np.save(os.path.join(val_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_val_preds.npy"), (np.array(best_val_scores) > best_threshold).astype(int))
        elif epoch >= min_epoch:
            counter += 1
            if counter >= patience:
                print(f"Early stopping at epoch {epoch} (no improvement after {patience} epochs)")
                break

    # Save metrics for this run
    all_val_metrics.append(best_val_metrics)

    run_metrics_path = os.path.join(val_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_metrics.txt")
    with open(run_metrics_path, 'w') as f:
        for k, v in best_val_metrics.items():
            if k in ["ACC", "PRE", "REC"]:
                f.write(f"{k}: {v * 100:.2f}%\n")
            else:
                f.write(f"{k}: {v:.3f}\n")
        f.write(f"Best Threshold: {best_threshold:.5f}\n")

    # Reload the best saved model before testing
    model.load_state_dict(torch.load(model_path, weights_only=True))

    # TESTING
    model.eval()
    test_scores, test_labels = evaluate_model(model, test_loader)
    test_threshold = best_threshold # reuse the one selected on validation
    test_metrics, _ = compute_metrics(test_labels, test_scores, test_threshold)

    all_test_metrics.append(test_metrics)

    test_metrics_path = os.path.join(test_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_test_metrics.txt")
    with open(test_metrics_path, 'w') as f:
        for k, v in test_metrics.items():
            if k in ["ACC", "PRE", "REC"]:
                f.write(f"{k}: {v * 100:.2f}%\n")
            else:
                f.write(f"{k}: {v:.3f}\n")

    test_preds = (np.array(test_scores) > test_threshold).astype(int)
    np.save(os.path.join(test_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_test_labels.npy"), test_labels)
    np.save(os.path.join(test_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_test_scores.npy"), test_scores)
    np.save(os.path.join(test_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_test_preds.npy"), test_preds)

    # TESTING MODEL A ON SET 2
    if modelAB == "A":
        model.eval()
        test_scores, test_labels = evaluate_model(model, test2_loader)
        test_threshold = best_threshold # reuse the one selected on validation
        test_metrics, _ = compute_metrics(test_labels, test_scores, test_threshold)

        all_test2_metrics.append(test_metrics)

        test_metrics_path = os.path.join(test_log_dir, f"{modelAB}_on_S2_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_test_metrics.txt")
        with open(test_metrics_path, 'w') as f:
            for k, v in test_metrics.items():
                if k in ["ACC", "PRE", "REC"]:
                    f.write(f"{k}: {v * 100:.2f}%\n")
                else:
                    f.write(f"{k}: {v:.3f}\n")

        test_preds = (np.array(test_scores) > test_threshold).astype(int)
        np.save(os.path.join(test_log_dir, f"{modelAB}_on_S2_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_test_labels.npy"), test_labels)
        np.save(os.path.join(test_log_dir, f"{modelAB}_on_S2_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_test_scores.npy"), test_scores)
        np.save(os.path.join(test_log_dir, f"{modelAB}_on_S2_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_test_preds.npy"), test_preds)

# AFTER ALL RUNS
# Compute mean and std for each metric
metrics_summary = {}
for key in all_val_metrics[0].keys():
    values = [metrics[key] for metrics in all_val_metrics]
    metrics_summary[key + '_mean'] = np.mean(values)
    metrics_summary[key + '_std'] = np.std(values)

metrics_summary_path = os.path.join(val_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_final_validation_summary.txt")
with open(metrics_summary_path, 'w') as f:
    for k, v in metrics_summary.items():
        f.write(f"{k}: {v:.3f}\n")

print("\n========== Final Averaged Validation Metrics ==========")
for k, v in metrics_summary.items():
    print(f"{k}: {v:.3f}")

# Compute mean and std for each metric
metrics_summary = {}
for key in all_test_metrics[0].keys():
    values = [metrics[key] for metrics in all_test_metrics]
    metrics_summary[key + '_mean'] = np.mean(values)
    metrics_summary[key + '_std'] = np.std(values)

metrics_summary_path = os.path.join(test_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_final_test_summary.txt")
with open(metrics_summary_path, 'w') as f:
    for k, v in metrics_summary.items():
        f.write(f"{k}: {v:.3f}\n")

print("\n========== Final Averaged test Metrics ==========")
for k, v in metrics_summary.items():
    print(f"{k}: {v:.3f}")

if modelAB == "A":
    # Compute mean and std for each metric
    metrics_summary = {}
    for key in all_test2_metrics[0].keys():
        values = [metrics[key] for metrics in all_test2_metrics]
        metrics_summary[key + '_mean'] = np.mean(values)
        metrics_summary[key + '_std'] = np.std(values)

    metrics_summary_path = os.path.join(test_log_dir, f"{modelAB}_on_S2_{'Bi-' if is_bilstm else ''}LSTM-AE_final_test_summary.txt")
    with open(metrics_summary_path, 'w') as f:
        for k, v in metrics_summary.items():
            f.write(f"{k}: {v:.3f}\n")

    print("\n========== Final Averaged test Metrics ==========")
    for k, v in metrics_summary.items():
        print(f"{k}: {v:.3f}")


========== Run 1 / 6 ==========
Early stopping at epoch 27 (no improvement after 10 epochs)

========== Run 2 / 6 ==========
Early stopping at epoch 48 (no improvement after 10 epochs)

========== Run 3 / 6 ==========
Early stopping at epoch 30 (no improvement after 10 epochs)

========== Run 4 / 6 ==========
Early stopping at epoch 67 (no improvement after 10 epochs)

========== Run 5 / 6 ==========
Early stopping at epoch 56 (no improvement after 10 epochs)

========== Run 6 / 6 ==========
Early stopping at epoch 24 (no improvement after 10 epochs)

========== Final Averaged Validation Metrics ==========
ACC_mean: 0.880
ACC_std: 0.029
PRE_mean: 0.515
PRE_std: 0.107
REC_mean: 0.404
REC_std: 0.150
MCC_mean: 0.378
MCC_std: 0.007
AUC_mean: 0.810
AUC_std: 0.016

========== Final Averaged test Metrics ==========
ACC_mean: 0.797
ACC_std: 0.012
PRE_mean: 0.290
PRE_std: 0.031
REC_mean: 0.566
REC_std: 0.176
MCC_mean: 0.298
MCC_std: 0.090
AUC_mean: 0.829
AUC_std: 0.043

========== Final Averag

In [ ]:
s2_val = pd.read_csv(os.path.join(target_dir, "S2_scaled_val.csv"))

# Perform multiple training + evaluation runs
all_val2_metrics = []

modelAB         = "A"
num_states      = 4
# 1. Reload best parameters
is_bilstm       = True
sequence_length = 14
dropout         = 0.4
embedding_dim   = 2
layer_sizes     = [256, 128, 64, 32, 16]
learning_rate   = 0.006161330774897911

for run_idx in range(NUM_RUNS):
    seed = 36 + (run_idx * 10)
    g = set_random_seeds(seed)

    val2_ds = WildfireDataset(s2_val, sequence_length, only_no_fire=False)
    val2_loader = DataLoader(val2_ds, batch_size=64, shuffle=False)

    model_path = os.path.join(val_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_model.pth")
    
    model = LSTMAutoencoder(
        input_dim=len(NUMERIC_FEATURES),
        embedding_dim=embedding_dim,
        layer_sizes=layer_sizes,
        num_states=num_states,
        bidirectional=is_bilstm,
        dropout=dropout
    )

    model.load_state_dict(torch.load(model_path, weights_only=True))
    model.eval()

    # Load best threshold from metrics file
    threshold_path = os.path.join(val_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_metrics.txt")
    with open(threshold_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            if "Best Threshold" in line:
                val_threshold = float(line.strip().split(":")[1])
                break

    val_scores, val_labels = evaluate_model(model, val2_loader)
    val_metrics, _ = compute_metrics(val_labels, val_scores, val_threshold)

    all_val2_metrics.append(val_metrics)

    val_metrics_path = os.path.join(val_log_dir, f"{modelAB}_on_S2_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_val_metrics.txt")
    with open(val_metrics_path, 'w') as f:
        for k, v in val_metrics.items():
            if k in ["ACC", "PRE", "REC"]:
                f.write(f"{k}: {v * 100:.2f}%\n")
            else:
                f.write(f"{k}: {v:.3f}\n")

    val_preds = (np.array(val_scores) > val_threshold).astype(int)
    np.save(os.path.join(val_log_dir, f"{modelAB}_on_S2_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_val_labels.npy"), val_labels)
    np.save(os.path.join(val_log_dir, f"{modelAB}_on_S2_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_val_scores.npy"), val_scores)
    np.save(os.path.join(val_log_dir, f"{modelAB}_on_S2_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_val_preds.npy"), val_preds)

# AFTER ALL RUNS
# Compute mean and std for each metric
metrics_summary = {}
for key in all_val2_metrics[0].keys():
    values = [metrics[key] for metrics in all_val2_metrics]
    metrics_summary[key + '_mean'] = np.mean(values)
    metrics_summary[key + '_std'] = np.std(values)

metrics_summary_path = os.path.join(val_log_dir, f"{modelAB}_on_S2_{'Bi-' if is_bilstm else ''}LSTM-AE_final_validation_summary.txt")
with open(metrics_summary_path, 'w') as f:
    for k, v in metrics_summary.items():
        f.write(f"{k}: {v:.3f}\n")

print("\n========== Final Averaged Validation Metrics ==========")
for k, v in metrics_summary.items():
    print(f"{k}: {v:.3f}")


========== Final Averaged Validation Metrics ==========
ACC_mean: 0.778
ACC_std: 0.073
PRE_mean: 0.439
PRE_std: 0.108
REC_mean: 0.239
REC_std: 0.263
MCC_mean: 0.159
MCC_std: 0.052
AUC_mean: 0.698
AUC_std: 0.020


#### **Models B: Validate and Test**

In [ ]:
s2_train = pd.read_csv(os.path.join(target_dir, "S2_scaled_train.csv"))
s2_val = pd.read_csv(os.path.join(target_dir, "S2_scaled_val.csv"))
s2_test = pd.read_csv(os.path.join(target_dir, "S2_scaled_test.csv"))
test_log_dir = os.path.join(target_dir, "00_3_testing")
val_log_dir = os.path.join(target_dir, "00_2_validation", "Validation_results")

NUM_RUNS = 6 # number of repetitions to create a distribution
def set_random_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    g = torch.Generator().manual_seed(seed)
    return g

smoothing_alpha = 0.1
patience, min_epoch = 10, 15

modelAB         = "B"
# 1. Reload best parameters
is_bilstm       = False
sequence_length = 30
dropout         = 0.1
embedding_dim   = 8
layer_sizes     = [256, 128, 64, 32, 16]
learning_rate   = 0.0008188148579607131

# Perform multiple training + evaluation runs
all_val_metrics   = []
all_test_metrics  = []
all_test2_metrics = []

for run_idx in range(NUM_RUNS):
    print(f"\n========== Run {run_idx + 1} / {NUM_RUNS} ==========")
    seed = 36 + (run_idx * 10)
    g = set_random_seeds(seed)

    # Prepare datasets
    train_ds = WildfireDataset(s2_train, sequence_length, only_no_fire=True)
    val_ds = WildfireDataset(s2_val, sequence_length, only_no_fire=False)
    train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, generator=g)
    val_loader = DataLoader(val_ds, batch_size=64, shuffle=False)
    test_ds = WildfireDataset(s2_test, sequence_length, only_no_fire=False)
    test_loader = DataLoader(test_ds, batch_size=64, shuffle=False)

    # Create a fresh model for each run
    model = LSTMAutoencoder(
        input_dim=len(NUMERIC_FEATURES),
        embedding_dim=embedding_dim,
        layer_sizes=layer_sizes,
        num_states=s1_train[CAT_FEATURE].nunique(),
        bidirectional=is_bilstm,
        dropout=dropout
    )
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    best_score, smoothed_score, smoothed_best_score = -float('inf'), None, -float('inf')
    counter = 0

    # TRAINING AND VALIDATION
    for epoch in range(1, 201):
        loss = train_one_epoch(model, train_loader, optimizer, criterion)
        val_scores, val_labels = evaluate_model(model, val_loader)
        threshold, _ = find_best_f1_threshold(val_labels, val_scores)
        val_metrics, _ = compute_metrics(val_labels, val_scores, threshold)

        val_loss = np.mean(val_scores)
        mcc = val_metrics["MCC"]
        rec = val_metrics["REC"]
        pre = val_metrics["PRE"]
        auc_score = val_metrics["AUC"]

        scaled_mcc = (mcc + 1) / 2
        scaled_auc = (auc_score - 0.5) * 2
        combo_score = 0.6 * scaled_mcc + 0.05 * rec + 0.05 * pre + 0.05 * scaled_auc

        if smoothed_score is None:
            smoothed_score = combo_score
        else:
            smoothed_score = (smoothing_alpha * combo_score + (1 - smoothing_alpha) * smoothed_score)

        if combo_score > best_score or smoothed_score > smoothed_best_score:
            smoothed_best_score = smoothed_score
            counter = 0
            if combo_score > best_score:
                best_score = combo_score
                best_val_scores = val_scores
                best_val_labels = val_labels
                best_threshold = threshold
                best_val_metrics = val_metrics
                model_path = os.path.join(val_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_model.pth")
                torch.save(model.state_dict(), model_path)
                np.save(os.path.join(val_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_val_labels.npy"), best_val_labels)
                np.save(os.path.join(val_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_val_scores.npy"), best_val_scores)
                np.save(os.path.join(val_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_val_preds.npy"), (np.array(best_val_scores) > best_threshold).astype(int))
        elif epoch >= min_epoch:
            counter += 1
            if counter >= patience:
                print(f"Early stopping at epoch {epoch} (no improvement after {patience} epochs)")
                break

    # Save metrics for this run
    all_val_metrics.append(best_val_metrics)

    run_metrics_path = os.path.join(val_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_metrics.txt")
    with open(run_metrics_path, 'w') as f:
        for k, v in best_val_metrics.items():
            if k in ["ACC", "PRE", "REC"]:
                f.write(f"{k}: {v * 100:.2f}%\n")
            else:
                f.write(f"{k}: {v:.3f}\n")
        f.write(f"Best Threshold: {best_threshold:.5f}\n")

    # Reload the best saved model before testing
    model.load_state_dict(torch.load(model_path, weights_only=True))

    # TESTING
    model.eval()
    test_scores, test_labels = evaluate_model(model, test_loader)
    test_threshold = best_threshold # reuse the one selected on validation
    test_metrics, _ = compute_metrics(test_labels, test_scores, test_threshold)

    all_test_metrics.append(test_metrics)

    test_metrics_path = os.path.join(test_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_test_metrics.txt")
    with open(test_metrics_path, 'w') as f:
        for k, v in test_metrics.items():
            if k in ["ACC", "PRE", "REC"]:
                f.write(f"{k}: {v * 100:.2f}%\n")
            else:
                f.write(f"{k}: {v:.3f}\n")

    test_preds = (np.array(test_scores) > test_threshold).astype(int)
    np.save(os.path.join(test_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_test_labels.npy"), test_labels)
    np.save(os.path.join(test_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_test_scores.npy"), test_scores)
    np.save(os.path.join(test_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_test_preds.npy"), test_preds)

    # TESTING MODEL A ON SET 2
    if modelAB == "A":
        model.eval()
        test_scores, test_labels = evaluate_model(model, test2_loader)
        test_threshold = best_threshold # reuse the one selected on validation
        test_metrics, _ = compute_metrics(test_labels, test_scores, test_threshold)

        all_test2_metrics.append(test_metrics)

        test_metrics_path = os.path.join(test_log_dir, f"{modelAB}_on_S2_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_test_metrics.txt")
        with open(test_metrics_path, 'w') as f:
            for k, v in test_metrics.items():
                if k in ["ACC", "PRE", "REC"]:
                    f.write(f"{k}: {v * 100:.2f}%\n")
                else:
                    f.write(f"{k}: {v:.3f}\n")

        test_preds = (np.array(test_scores) > test_threshold).astype(int)
        np.save(os.path.join(test_log_dir, f"{modelAB}_on_S2_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_test_labels.npy"), test_labels)
        np.save(os.path.join(test_log_dir, f"{modelAB}_on_S2_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_test_scores.npy"), test_scores)
        np.save(os.path.join(test_log_dir, f"{modelAB}_on_S2_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_test_preds.npy"), test_preds)

# AFTER ALL RUNS
# Compute mean and std for each metric
metrics_summary = {}
for key in all_val_metrics[0].keys():
    values = [metrics[key] for metrics in all_val_metrics]
    metrics_summary[key + '_mean'] = np.mean(values)
    metrics_summary[key + '_std'] = np.std(values)

metrics_summary_path = os.path.join(val_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_final_validation_summary.txt")
with open(metrics_summary_path, 'w') as f:
    for k, v in metrics_summary.items():
        f.write(f"{k}: {v:.3f}\n")

print("\n========== Final Averaged Validation Metrics ==========")
for k, v in metrics_summary.items():
    print(f"{k}: {v:.3f}")

# Compute mean and std for each metric
metrics_summary = {}
for key in all_test_metrics[0].keys():
    values = [metrics[key] for metrics in all_test_metrics]
    metrics_summary[key + '_mean'] = np.mean(values)
    metrics_summary[key + '_std'] = np.std(values)

metrics_summary_path = os.path.join(test_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_final_test_summary.txt")
with open(metrics_summary_path, 'w') as f:
    for k, v in metrics_summary.items():
        f.write(f"{k}: {v:.3f}\n")

print("\n========== Final Averaged test Metrics ==========")
for k, v in metrics_summary.items():
    print(f"{k}: {v:.3f}")

if modelAB == "A":
    # Compute mean and std for each metric
    metrics_summary = {}
    for key in all_test2_metrics[0].keys():
        values = [metrics[key] for metrics in all_test2_metrics]
        metrics_summary[key + '_mean'] = np.mean(values)
        metrics_summary[key + '_std'] = np.std(values)

    metrics_summary_path = os.path.join(test_log_dir, f"{modelAB}_on_S2_{'Bi-' if is_bilstm else ''}LSTM-AE_final_test_summary.txt")
    with open(metrics_summary_path, 'w') as f:
        for k, v in metrics_summary.items():
            f.write(f"{k}: {v:.3f}\n")

    print("\n========== Final Averaged test Metrics ==========")
    for k, v in metrics_summary.items():
        print(f"{k}: {v:.3f}")


========== Run 1 / 6 ==========
Early stopping at epoch 24 (no improvement after 10 epochs)

========== Run 2 / 6 ==========
Early stopping at epoch 30 (no improvement after 10 epochs)

========== Run 3 / 6 ==========
Early stopping at epoch 24 (no improvement after 10 epochs)

========== Run 4 / 6 ==========
Early stopping at epoch 44 (no improvement after 10 epochs)

========== Run 5 / 6 ==========
Early stopping at epoch 24 (no improvement after 10 epochs)

========== Run 6 / 6 ==========
Early stopping at epoch 24 (no improvement after 10 epochs)

========== Final Averaged Validation Metrics ==========
ACC_mean: 0.686
ACC_std: 0.068
PRE_mean: 0.333
PRE_std: 0.037
REC_mean: 0.793
REC_std: 0.073
MCC_mean: 0.355
MCC_std: 0.029
AUC_mean: 0.778
AUC_std: 0.020

========== Final Averaged test Metrics ==========
ACC_mean: 0.659
ACC_std: 0.060
PRE_mean: 0.310
PRE_std: 0.032
REC_mean: 0.883
REC_std: 0.041
MCC_mean: 0.371
MCC_std: 0.035
AUC_mean: 0.817
AUC_std: 0.017


In [ ]:
s2_train = pd.read_csv(os.path.join(target_dir, "S2_scaled_train.csv"))
s2_val = pd.read_csv(os.path.join(target_dir, "S2_scaled_val.csv"))
s2_test = pd.read_csv(os.path.join(target_dir, "S2_scaled_test.csv"))
test_log_dir = os.path.join(target_dir, "00_3_testing")
val_log_dir = os.path.join(target_dir, "00_2_validation", "Validation_results")

NUM_RUNS = 6 # number of repetitions to create a distribution
def set_random_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    g = torch.Generator().manual_seed(seed)
    return g

smoothing_alpha = 0.1
patience, min_epoch = 10, 15

modelAB         = "B"
# 1. Reload best parameters
is_bilstm       = True
sequence_length = 14
dropout         = 0.4
embedding_dim   = 2
layer_sizes     = [256, 128, 64, 32, 16]
learning_rate   = 0.006161330774897911

# Perform multiple training + evaluation runs
all_val_metrics   = []
all_test_metrics  = []
all_test2_metrics = []

for run_idx in range(NUM_RUNS):
    print(f"\n========== Run {run_idx + 1} / {NUM_RUNS} ==========")
    seed = 36 + (run_idx * 10)
    g = set_random_seeds(seed)

    # Prepare datasets
    train_ds = WildfireDataset(s2_train, sequence_length, only_no_fire=True)
    val_ds = WildfireDataset(s2_val, sequence_length, only_no_fire=False)
    train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, generator=g)
    val_loader = DataLoader(val_ds, batch_size=64, shuffle=False)
    test_ds = WildfireDataset(s2_test, sequence_length, only_no_fire=False)
    test_loader = DataLoader(test_ds, batch_size=64, shuffle=False)

    # Create a fresh model for each run
    model = LSTMAutoencoder(
        input_dim=len(NUMERIC_FEATURES),
        embedding_dim=embedding_dim,
        layer_sizes=layer_sizes,
        num_states=s1_train[CAT_FEATURE].nunique(),
        bidirectional=is_bilstm,
        dropout=dropout
    )
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    best_score, smoothed_score, smoothed_best_score = -float('inf'), None, -float('inf')
    counter = 0

    # TRAINING AND VALIDATION
    for epoch in range(1, 201):
        loss = train_one_epoch(model, train_loader, optimizer, criterion)
        val_scores, val_labels = evaluate_model(model, val_loader)
        threshold, _ = find_best_f1_threshold(val_labels, val_scores)
        val_metrics, _ = compute_metrics(val_labels, val_scores, threshold)

        val_loss = np.mean(val_scores)
        mcc = val_metrics["MCC"]
        rec = val_metrics["REC"]
        pre = val_metrics["PRE"]
        auc_score = val_metrics["AUC"]

        scaled_mcc = (mcc + 1) / 2
        scaled_auc = (auc_score - 0.5) * 2
        combo_score = 0.6 * scaled_mcc + 0.05 * rec + 0.05 * pre + 0.05 * scaled_auc

        if smoothed_score is None:
            smoothed_score = combo_score
        else:
            smoothed_score = (smoothing_alpha * combo_score + (1 - smoothing_alpha) * smoothed_score)

        if combo_score > best_score or smoothed_score > smoothed_best_score:
            smoothed_best_score = smoothed_score
            counter = 0
            if combo_score > best_score:
                best_score = combo_score
                best_val_scores = val_scores
                best_val_labels = val_labels
                best_threshold = threshold
                best_val_metrics = val_metrics
                model_path = os.path.join(val_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_model.pth")
                torch.save(model.state_dict(), model_path)
                np.save(os.path.join(val_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_val_labels.npy"), best_val_labels)
                np.save(os.path.join(val_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_val_scores.npy"), best_val_scores)
                np.save(os.path.join(val_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_val_preds.npy"), (np.array(best_val_scores) > best_threshold).astype(int))
        elif epoch >= min_epoch:
            counter += 1
            if counter >= patience:
                print(f"Early stopping at epoch {epoch} (no improvement after {patience} epochs)")
                break

    # Save metrics for this run
    all_val_metrics.append(best_val_metrics)

    run_metrics_path = os.path.join(val_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_metrics.txt")
    with open(run_metrics_path, 'w') as f:
        for k, v in best_val_metrics.items():
            if k in ["ACC", "PRE", "REC"]:
                f.write(f"{k}: {v * 100:.2f}%\n")
            else:
                f.write(f"{k}: {v:.3f}\n")
        f.write(f"Best Threshold: {best_threshold:.5f}\n")

    # Reload the best saved model before testing
    model.load_state_dict(torch.load(model_path, weights_only=True))

    # TESTING
    model.eval()
    test_scores, test_labels = evaluate_model(model, test_loader)
    test_threshold = best_threshold # reuse the one selected on validation
    test_metrics, _ = compute_metrics(test_labels, test_scores, test_threshold)

    all_test_metrics.append(test_metrics)

    test_metrics_path = os.path.join(test_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_test_metrics.txt")
    with open(test_metrics_path, 'w') as f:
        for k, v in test_metrics.items():
            if k in ["ACC", "PRE", "REC"]:
                f.write(f"{k}: {v * 100:.2f}%\n")
            else:
                f.write(f"{k}: {v:.3f}\n")

    test_preds = (np.array(test_scores) > test_threshold).astype(int)
    np.save(os.path.join(test_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_test_labels.npy"), test_labels)
    np.save(os.path.join(test_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_test_scores.npy"), test_scores)
    np.save(os.path.join(test_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_test_preds.npy"), test_preds)

    # TESTING MODEL A ON SET 2
    if modelAB == "A":
        model.eval()
        test_scores, test_labels = evaluate_model(model, test2_loader)
        test_threshold = best_threshold # reuse the one selected on validation
        test_metrics, _ = compute_metrics(test_labels, test_scores, test_threshold)

        all_test2_metrics.append(test_metrics)

        test_metrics_path = os.path.join(test_log_dir, f"{modelAB}_on_S2_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_test_metrics.txt")
        with open(test_metrics_path, 'w') as f:
            for k, v in test_metrics.items():
                if k in ["ACC", "PRE", "REC"]:
                    f.write(f"{k}: {v * 100:.2f}%\n")
                else:
                    f.write(f"{k}: {v:.3f}\n")

        test_preds = (np.array(test_scores) > test_threshold).astype(int)
        np.save(os.path.join(test_log_dir, f"{modelAB}_on_S2_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_test_labels.npy"), test_labels)
        np.save(os.path.join(test_log_dir, f"{modelAB}_on_S2_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_test_scores.npy"), test_scores)
        np.save(os.path.join(test_log_dir, f"{modelAB}_on_S2_{'Bi-' if is_bilstm else ''}LSTM-AE_run_{run_idx+1}_test_preds.npy"), test_preds)

# AFTER ALL RUNS
# Compute mean and std for each metric
metrics_summary = {}
for key in all_val_metrics[0].keys():
    values = [metrics[key] for metrics in all_val_metrics]
    metrics_summary[key + '_mean'] = np.mean(values)
    metrics_summary[key + '_std'] = np.std(values)

metrics_summary_path = os.path.join(val_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_final_validation_summary.txt")
with open(metrics_summary_path, 'w') as f:
    for k, v in metrics_summary.items():
        f.write(f"{k}: {v:.3f}\n")

print("\n========== Final Averaged Validation Metrics ==========")
for k, v in metrics_summary.items():
    print(f"{k}: {v:.3f}")

# Compute mean and std for each metric
metrics_summary = {}
for key in all_test_metrics[0].keys():
    values = [metrics[key] for metrics in all_test_metrics]
    metrics_summary[key + '_mean'] = np.mean(values)
    metrics_summary[key + '_std'] = np.std(values)

metrics_summary_path = os.path.join(test_log_dir, f"{modelAB}_{'Bi-' if is_bilstm else ''}LSTM-AE_final_test_summary.txt")
with open(metrics_summary_path, 'w') as f:
    for k, v in metrics_summary.items():
        f.write(f"{k}: {v:.3f}\n")

print("\n========== Final Averaged test Metrics ==========")
for k, v in metrics_summary.items():
    print(f"{k}: {v:.3f}")

if modelAB == "A":
    # Compute mean and std for each metric
    metrics_summary = {}
    for key in all_test2_metrics[0].keys():
        values = [metrics[key] for metrics in all_test2_metrics]
        metrics_summary[key + '_mean'] = np.mean(values)
        metrics_summary[key + '_std'] = np.std(values)

    metrics_summary_path = os.path.join(test_log_dir, f"{modelAB}_on_S2_{'Bi-' if is_bilstm else ''}LSTM-AE_final_test_summary.txt")
    with open(metrics_summary_path, 'w') as f:
        for k, v in metrics_summary.items():
            f.write(f"{k}: {v:.3f}\n")

    print("\n========== Final Averaged test Metrics ==========")
    for k, v in metrics_summary.items():
        print(f"{k}: {v:.3f}")


========== Run 1 / 6 ==========
Early stopping at epoch 24 (no improvement after 10 epochs)

========== Run 2 / 6 ==========
Early stopping at epoch 24 (no improvement after 10 epochs)

========== Run 3 / 6 ==========
Early stopping at epoch 29 (no improvement after 10 epochs)

========== Run 4 / 6 ==========
Early stopping at epoch 52 (no improvement after 10 epochs)

========== Run 5 / 6 ==========
Early stopping at epoch 43 (no improvement after 10 epochs)

========== Run 6 / 6 ==========
Early stopping at epoch 81 (no improvement after 10 epochs)

========== Final Averaged Validation Metrics ==========
ACC_mean: 0.725
ACC_std: 0.055
PRE_mean: 0.335
PRE_std: 0.042
REC_mean: 0.568
REC_std: 0.102
MCC_mean: 0.274
MCC_std: 0.039
AUC_mean: 0.702
AUC_std: 0.035

========== Final Averaged test Metrics ==========
ACC_mean: 0.685
ACC_std: 0.067
PRE_mean: 0.296
PRE_std: 0.055
REC_mean: 0.625
REC_std: 0.162
MCC_mean: 0.255
MCC_std: 0.097
AUC_mean: 0.726
AUC_std: 0.055
